# Using JKQ DDSIM from Python

JKQ DDSIM is available for multiple Python versions (>=3.6) from [PyPI](https://pypi.org/project/jkq.ddsim/). 
Using it as backend for [Qiskit](https://qiskit.org/) additionally requires at least [qiskit-terra](https://pypi.org/project/qiskit-terra/).

In a [virtual environment](https://docs.python.org/3/tutorial/venv.html) you can use the following snippet:
```
(venv) $ pip install jkq.ddsim qiskit-terra
```

The JKQProvider currently has two backends
- QasmSimulator simulates a circuit and generates the given number of shots
- StatevectorSimulator simulates the circuit and returns the statevector

## QasmSimulator for Sampling

The QasmSimulator-Backend take a QuantumCircuit object and simulates it using decision diagrams in the underlying C++ implementation.
For circuits with no non-unitary operations (except for measurements at the end of the circuit) the simulation is only done once and the samples subsequently drawn from the decision diagram, resulting in fast runtime.

In [1]:
from qiskit import *

from jkq import ddsim

# Circuit to create a Bell state
circ = QuantumCircuit(3)
circ.h(0)
circ.cx(0, 1)
circ.cx(0, 2)


# Show circuit
print(circ.draw(fold=-1))

provider = ddsim.JKQProvider()

# get the QasmSimulator and sample 100000 times
backend = provider.get_backend('qasm_simulator')
print(f'Backend version: {backend.configuration().backend_version}')
job = execute(circ, backend, shots=100000)
result = job.result()
counts = result.get_counts(circ)
print(counts)

     ┌───┐          
q_0: ┤ H ├──■────■──
     └───┘┌─┴─┐  │  
q_1: ─────┤ X ├──┼──
          └───┘┌─┴─┐
q_2: ──────────┤ X ├
               └───┘
Backend version: 1.6.0
{'000': 49709, '111': 50291}


## StatevectorSimulator for Observing the Statevector

The StatevectorSimulator-Backend takes a QuantumCircuit as above but return the state vector instead of a number of samples.

In [2]:
# get the StatevectorSimulator and calculate the statevector
backend = provider.get_backend('statevector_simulator')
print(f'Backend version: {backend.configuration().backend_version}')
job = execute(circ, backend)
result = job.result()
counts = result.get_statevector(circ)
print(counts)

Backend version: 1.6.0
[0.70710678+0.j 0.        +0.j 0.        +0.j 0.        +0.j
 0.        +0.j 0.        +0.j 0.        +0.j 0.70710678+0.j]
